# Spherical harmonic analysis on a grid
In this example, a sample of fully normalized spherical harmonic coefficients will be evaluated (analyzed) on a grid and a set of points.

In [1]:
%load_ext autoreload 
%autoreload 2
%load_ext Cython

from frommle2.sh.analysis import Analysis
from frommle2.sh.ynm import Ynm
import frommle2.sh.xarraysh
import xarray as xr
import numpy as np

## Load a test dataset expressed in fully normalized spherical harmonic coefficients
In this example it is a spherical harmonic expansion of the Rhine watershed

In [2]:
## Let's load the test dataset
basinfile="data/rhinemask.nc"
dsrhine=xr.open_dataset(basinfile)
# To access nmax, nim etc we need to build a pandas multindex from the provided triplets of degree (n), order(m) and trigonometric sign (t)
dsrhine=dsrhine.sh.build_MultiIndex()

shana=Analysis(dsrhine.sh.nmax)

display(shana(dsrhine.cnm))

ValueError: cannot add coordinates with new dimensions to a DataArray

In [13]:
crds=dsrhine.cnm.drop_vars("shg").coords
display(crds)

Coordinates:
    name     <U20 'RHINE               '

In [4]:
#%%cython
#cimport numpy as np
ynm=Ynm(15)

def testynm():
    for lat in np.arange(-90,90.0,0.5):
        for lon in np.arange(-180.0,180.0,0.5):
            tmp=ynm(lon,lat)

In [5]:
%prun testynm()

         365 function calls in 0.306 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.304    0.304    0.306    0.306 604551106.py:5(testynm)
      361    0.002    0.000    0.002    0.000 {built-in method numpy.arange}
        1    0.000    0.000    0.306    0.306 {built-in method builtins.exec}
        1    0.000    0.000    0.306    0.306 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}

In [9]:
import frommle2.sh.xarraysh
mi=xr.DataArray.sh.mi_fromarrays(ynm.nmt().T)
# ynm.nmax
display(mi)

MultiIndex([( 0,  0, 0),
            ( 0,  0, 1),
            ( 1,  0, 0),
            ( 1,  0, 1),
            ( 2,  0, 0),
            ( 2,  0, 1),
            ( 3,  0, 0),
            ( 3,  0, 1),
            ( 4,  0, 0),
            ( 4,  0, 1),
            ...
            (14, 13, 0),
            (14, 13, 1),
            (15, 13, 0),
            (15, 13, 1),
            (14, 14, 0),
            (14, 14, 1),
            (15, 14, 0),
            (15, 14, 1),
            (15, 15, 0),
            (15, 15, 1)],
           names=['n', 'm', 't'], length=272)

In [3]:
%%cython
from frommle2.sh.ynm cimport Ynm 


Error compiling Cython file:
------------------------------------------------------------
...
from frommle2.sh.ynm cimport Ynm 
^
------------------------------------------------------------

/home/roelof/.cache/ipython/cython/_cython_magic_c7abf65a2245de2f9c2b12cb2204cc5c.pyx:1:0: 'frommle2/sh/ynm.pxd' not found

Error compiling Cython file:
------------------------------------------------------------
...
from frommle2.sh.ynm cimport Ynm 
^
------------------------------------------------------------

/home/roelof/.cache/ipython/cython/_cython_magic_c7abf65a2245de2f9c2b12cb2204cc5c.pyx:1:0: 'frommle2/sh/ynm/Ynm.pxd' not found


## Generate a forward operator which will map the input coefficients to a grid

In [3]:
# Set up a grid
west=-10
east=20
south=30
north=55
dres=0.25
lon=np.arange(west,east,dres)
lat=np.arange(south,north,dres)

ynm=Ynm(dsrhine.sh.nmax)#,lon=lon,lat=lat)

In [6]:
## Apply the forward operator to the input spherical harmonic coefficients

# ynm.dsmapper.lon
# rhinegrid=ynm(dsrhine.cnm)
display(rhinegrid)

NameError: name 'rhinegrid' is not defined

In [48]:
nlon=ynm.dsmapper.dims["lon"]
nmax=ynm.dsmapper.sh.nmax
mlon=np.array([(m,m*lon)  for m in range(0,nmax+1) for lon in np.deg2rad(ynm.dsmapper.lon.values)]).reshape([nmax+1,nlon])


ValueError: cannot reshape array of size 29040 into shape (121,120)

In [1]:
%load_ext autoreload 
%autoreload 2
# %load_ext Cython

from frommle2.io.shascii import readSHAscii
from io import StringIO


unitvaldate=""" META    5    0.000000    0.000000    0.000000
     0    0  0.10000000000000E+01  0.00000000000000E+00
     1    0  0.13832772801306E+01  0.00000000000000E+00
     2    0  0.10212748929338E+01  0.00000000000000E+00
     3    0  0.19977631019326E+00  0.00000000000000E+00
     4    0 -0.71104388667820E+00  0.00000000000000E+00
     5    0 -0.13303831637421E+01  0.00000000000000E+00
     1    1  0.10423345064422E+01  0.90963154314668E-02
     2    1  0.18614043905837E+01  0.16244230021762E-01
     3    1  0.21343981280424E+01  0.18626610276269E-01
     4    1  0.16696066487732E+01  0.14570436486416E-01
     5    1  0.59290758193332E+00  0.51742260796706E-02
     2    2  0.70125429368904E+00  0.12240439227534E-01
     3    2  0.14817439746500E+01  0.25863937284512E-01
     4    2  0.21041449543880E+01  0.36727986797224E-01
     5    2  0.22445390503895E+01  0.39178574858469E-01
     3    3  0.45575270596480E+00  0.11934304613339E-01
     4    3  0.10919408843509E+01  0.28593478355801E-01
     5    3  0.17913385584263E+01  0.46907850994811E-01
     4    4  0.29083907068701E+00  0.10156324146655E-01
     5    4  0.77036706491121E+00  0.26901810697799E-01
     5    5  0.18351146735867E+00  0.80122836994040E-02"""

fid = StringIO(unitvaldate)

shds=readSHAscii(fid)
display(shds)



<xarray.Dataset>
Dimensions:  (shg: 42)
Coordinates:
  * shg      (shg) MultiIndex
  - n        (shg) int64 0 0 1 1 2 2 3 3 4 4 5 5 1 ... 5 3 3 4 4 5 5 4 4 5 5 5 5
  - m        (shg) int64 0 0 0 0 0 0 0 0 0 0 0 0 1 ... 2 3 3 3 3 3 3 4 4 4 4 5 5
  - t        (shg) int64 0 1 0 1 0 1 0 1 0 1 0 1 0 ... 1 0 1 0 1 0 1 0 1 0 1 0 1
Data variables:
    cnm      (shg) float64 1.0 0.0 1.383 0.0 ... 0.7704 0.0269 0.1835 0.008012
Attributes:
    nmax:      5
    nmaxfile:  5
    CTime:     0001-01-01 00:00:00
    STime:     0001-01-01 00:00:00
    ETime:     0001-01-01 00:00:00

In [2]:
from frommle2.sh.isoload import unit as shunit

ds=shunit(20,[0.5],[53.0])
display(ds)

<xarray.DataArray (shg: 462, np: 1)>
array([[ 1.00000000e+00],
       [ 0.00000000e+00],
       [ 1.38327728e+00],
       [ 0.00000000e+00],
       [ 1.02127489e+00],
       [ 0.00000000e+00],
       [ 1.99776310e-01],
       [ 0.00000000e+00],
       [-7.11043887e-01],
       [-0.00000000e+00],
       [-1.33038316e+00],
       [-0.00000000e+00],
       [-1.40545088e+00],
       [-0.00000000e+00],
       [-9.07535168e-01],
       [-0.00000000e+00],
       [-4.04503985e-02],
       [-0.00000000e+00],
       [ 8.43398097e-01],
       [ 0.00000000e+00],
...
       [ 5.43409918e-04],
       [ 8.12131564e-05],
       [ 2.63983656e-03],
       [ 3.94526217e-04],
       [ 9.03833496e-03],
       [ 1.35078821e-03],
       [ 2.51587311e-02],
       [ 3.75999755e-03],
       [ 3.31098236e-04],
       [ 5.24408088e-05],
       [ 1.65134489e-03],
       [ 2.61547336e-04],
       [ 5.80609420e-03],
       [ 9.19594980e-04],
       [ 2.01578014e-04],
       [ 3.37325909e-05],
       [ 1.03082207e-03],
       [ 1.72500454e-04],
       [ 1.22635692e-04],
       [ 2.16239813e-05]])
Coordinates:
    lon      (np) float64 0.5
    lat      (np) float64 53.0
  * shg      (shg) MultiIndex
  - n        (shg) int64 0 0 1 1 2 2 3 3 4 4 5 ... 19 19 20 20 19 19 20 20 20 20
  - m        (shg) int64 0 0 0 0 0 0 0 0 0 0 0 ... 18 18 18 18 19 19 19 19 20 20
  - t        (shg) int64 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1
Dimensions without coordinates: np

In [10]:
from datetime import datetime

print(datetime.min)

0001-01-01 00:00:00


In [76]:
display(np.cos(dsuns.m*np.deg2rad(dsuns.lon)))

<xarray.DataArray (m: 121, lon: 120)>
array([[ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00, ...,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+00],
       [ 9.84807753e-01,  9.85556059e-01,  9.86285602e-01, ...,
         9.44089020e-01,  9.42641491e-01,  9.41176015e-01],
       [ 9.39692621e-01,  9.42641491e-01,  9.45518576e-01, ...,
         7.82608157e-01,  7.77145961e-01,  7.71624583e-01],
       ...,
       [-1.73648178e-01,  3.33806859e-01,  7.54709580e-01, ...,
        -3.66501227e-01, -7.77145961e-01, -9.86285602e-01],
       [-3.42020143e-01,  1.69349504e-01,  6.36078220e-01, ...,
        -6.52759752e-01, -9.42641491e-01, -9.84040698e-01],
       [-5.00000000e-01,  2.57237726e-15,  5.00000000e-01, ...,
        -8.66025404e-01, -1.00000000e+00, -8.66025404e-01]])
Coordinates:
  * m        (m) int64 0 1 2 3 4 5 6 7 8 ... 112 113 114 115 116 117 118 119 120
  * lon      (lon) float64 -10.0 -9.75 -9.5 -9.25 -9.0 ... 19.0 19.25 19.5 19.75